# <center>Cats Vs Dogs</center>




### <center>In this notebook, we will write a convolutional neural network(CNN) to classify images containing a dog or a cat. It is easy for people, dogs and cats. However, the computer will be a little more difficult</center>

![](https://pixelz.cc/wp-content/uploads/2018/10/dogs-and-cats-uhd-4k-wallpaper-768x432.jpg)



----------------------------


First we import the necessary libraries

In [ ]:
import os
import zipfile
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

# <center>Loading data</center>

To begin with, we will create a **path** variable that will be responsible for the path to our data. This is done for more convenient and faster path output.

Next, using **listdir**, we will load the list of image names from the path directory

In [ ]:
import zipfile
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip",'r') as z:
    z.extractall(".")

In [ ]:
path = '/kaggle/working/train/'
filenames = os.listdir(path)
filenames[:5]

Since in the future I will be converting our data to a **Dataframe**, we will create a target variable responsible for the picture class: **cat** and **dog**

In [ ]:
label = []
for filename in filenames:
    if filename.split('.')[0] =='cat':
        label.append('cat')
    else:
        label.append('dog')

In [ ]:
df = pd.DataFrame({
                   'name':filenames,
                   'label':label
                 })

In [ ]:
df.head()

Let's see on the histogram whether the **label**  is correct sorted all the photos into classes

In [ ]:
print(df['label'].value_counts())
sns.countplot(data=df, x=df['label']);

Great, we have an equal number of cat and dog classes

Let's extract a couple of images from the data (I manually looked at the images in the directory and chose the most presentable animals in the photos)

In [ ]:
load_img(path+'cat.10009.jpg')

In [ ]:
load_img(path+'dog.1283.jpg')

# <center>Base model</center>

First, let's write the basic architecture of **CNN**

In this model, we use 5 consecutive blocks from **Conv2D** and **MaxPooling2d** with different filter depths

Since we have a classification task, after 5 blocks we will transform our data into a 1D tensor and apply Dense layers,

the last **Dense layer** should have 1 layer and **activation='sigmoid'** since we have a binary classification task

**input_shape** we will set **(256, 256, 3)** as the base for further use of the generator without resizing the image


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()


----------------

Great, using **optimizer='Adam'** as the most basic and recommended

**loss='binary_crossentropy'** since we have a binary classification

**metrics='acc'** since we have the same number of classes and accuracy is a suitable metric

In [ ]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics='acc')

# <center>Data preprocessing: splitting data into train, test, val</center>

Since the competition has already ended and I will not be able to submit the test data that is attached to the dataset, so we will divide our sample into 3 parts: **train, test, val**.

We will use proportions **train:test:val** - **8:1:1**

On the **train** sample, we will train our model

On the **val** sample, we will check the ability of our model to generalize to unknown data

On the **test** sample, we will make the final prediction

-----------------

To get an equal number of classes when splitting the data, I will use **stratify**. With it, our classes will be related as **1:1**

In [ ]:
train, test_val = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=17)

In [ ]:
test, val = train_test_split(test_val, test_size=0.5,  stratify=test_val['label'], random_state=17)

In [ ]:
print('train size:', train.shape[0],
      '\nvalidation size:', val.shape[0],
      '\ntest size:', test.shape[0],     
     )

In [ ]:
print('train labels:\n',train['label'].value_counts(),
      '\n\nvalidataion labels:\n',val['label'].value_counts(),
      '\n\ntest labels:\n',test['label'].value_counts(),
      sep='')

Great, we see that all the data is split in a ratio of 8:1:1 and with the same ratio of classes

#  <center>Data Preprocessing: Data normalization</center>

Let's apply an **ImageDataGenerator** to our data to make it look like an **input_shape** for our model.

Neural networks need to receive scaled data as input, for this we apply **rescale=1./255**

In this case, the image size is not specified, because **flow_from_dataframe** creates **target_size=(256, 256)** and default **color_mode='rgb'**

in case of changing the size of the input tensors and using other values, we would have to manually specify the dimensions and depth of the image

----------------

Usually at this stage, the process of image **augmentation** is done.

Augmentation is the process of generating artificial images using rotations, mirroring, shifts and many other different methods based on existing ones.

This is one of the methods of dealing with overfitting of the model, if necessary, I will apply it in the future.



In [ ]:
train_gen = ImageDataGenerator(rescale=1./255)
train_data = train_gen.flow_from_dataframe(train,
                                           directory=path,
                                           x_col='name',
                                           y_col='label',
                                           class_mode='binary',
                                           seed=17                                          
                                          )

val_gen = ImageDataGenerator(rescale=1./255)
val_data = val_gen.flow_from_dataframe(val,
                                       directory=path,
                                       x_col='name',
                                       y_col='label',
                                       class_mode='binary',
                                       seed=17  
                                      )

I would also like to say about **batch_size**

In this case, I decided to use the standard **batch_size=32**, but at the expense of this, use fewer epochs for training.

The larger the batch_size value, the less time it takes to train one epoch and the more epochs are needed to get good results.

The number of iterations in one epoch is **train_size(20000)/batch_size(32)=625**

# <center>Base model training</center>

In [ ]:
history = model.fit(train_data,
                    validation_data = val_data,
                    epochs=10
                   )

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(15,8))
plt.plot(loss, label='Train loss')
plt.plot(val_loss,'--', label='Val loss')
plt.title('Training and validation loss')
plt.xticks(np.arange(0,10))
plt.yticks(np.arange(0, 0.7, 0.05));
plt.grid()
plt.legend();

The plot clearly shows that after the **5th** epoch, val loss stopped decreasing, but went up, we see the problem of **overfitting**

First, let's start expanding our data with **augmentation**.

#  <center>Data Preprocessing : Augmentation</center>

As I wrote above, augmentation is the process of generating artificial images using rotations, mirroring, shifts and other methods based on existing data.

Let me show examples of augmentation with photos of cats and dogs that I showed above.

In [ ]:
aug_gen = ImageDataGenerator(rescale = 1./255,
                               shear_range = 0.2,
                               zoom_range = 0.2,
                               rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               horizontal_flip=True,
                               fill_mode='nearest'
                              )

In [ ]:
img_cat = load_img(path+'cat.10009.jpg')
img_dog = load_img(path+'dog.1283.jpg')

img_cat_arr = image.img_to_array(img_cat)
img_cat_arr = img_cat_arr.reshape((1,)+img_cat_arr.shape)

img_dog_arr = image.img_to_array(img_dog)
img_dog_arr = img_dog_arr.reshape((1,)+ img_dog_arr.shape)

In [ ]:
aug_images_cat = aug_gen.flow(img_cat_arr, batch_size=1)
aug_images_dog = aug_gen.flow(img_dog_arr, batch_size=1)

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(141)
plt.imshow(img_cat)
plt.title("original")
i=2
for batch in aug_images_cat:
    plt.subplot(14*10+i)
    plt.imshow(image.array_to_img(batch[0]))
    plt.title("augmented")
    i += 1
    if i % 5 == 0:
        break

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(141)
plt.imshow(img_dog)
plt.title("original")
i=2
for batch in aug_images_dog:
    plt.subplot(14*10+i)
    plt.imshow(image.array_to_img(batch[0]))
    plt.title("augmented")
    i += 1
    if i % 5 == 0:
        break

Great, now let's apply augmentation to our **train data**.

It is important to apply **augmentation only to train data**, not to val and test. We need to provide our model with a large sample of images so that it can learn to recognize patterns in images.

Also in the new network, I'm going to change the size of incoming images to **(224x224)**, this size is most often used as input in many CNN networks, let's do the same.

Since we decided to change the input image size, we also need to change the generation for val_data

In [ ]:
train_data = aug_gen.flow_from_dataframe(train,
                                         directory=path,
                                         x_col='name',
                                         y_col='label',
                                         class_mode='binary',
                                         target_size=(224,224),
                                         seed=17
                                        )

val_data = val_gen.flow_from_dataframe(val,
                                       directory=path,
                                       x_col='name',
                                       y_col='label',
                                       class_mode='binary',
                                       target_size=(224,224),
                                       seed=17  
                                      )

Ok, now let's improve our model

#  <center>Model tuning</center>

Our base model looked like this

In [ ]:
model.summary()

It is important to understand that a strong fight against an **overfitting** problem can turn into an **underfitting** problem.

Perhaps the augmentation is already enough to solve the problem of overfitting our model, but I want to add some more examples of hyperparameter regularization:


- Let's try to apply **l2 regularization** with a small coefficient to the Dense layer


- Add a **Dropout layer** with a small value before the last output layer

In [ ]:
best_model = models.Sequential()

best_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
best_model.add(layers.MaxPooling2D((2, 2)))

best_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
best_model.add(layers.MaxPooling2D((2, 2)))

best_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
best_model.add(layers.MaxPooling2D((2, 2)))

best_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
best_model.add(layers.MaxPooling2D((2, 2)))

best_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
best_model.add(layers.MaxPooling2D((2, 2)))

best_model.add(layers.Flatten())
best_model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
best_model.add(layers.Dropout(0.2))
best_model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
best_model.summary()

Let's try to directly set the **learning_rate** to the optimizer, take a value slightly less than the default.

Thus, we guarantee that our optimizer will not get stuck in the local minimum of the function, however, for this we reduce the **learning rate**

In [ ]:
best_model.compile(optimizer=optimizers.Adam(learning_rate=5e-4), loss='binary_crossentropy', metrics='acc')

#  <center>Tuned model training</center>

Let's increase the number of **epochs to 60**, most likely the model will need many more epochs than in the previous version

We will also add **callbacks** that will stop training if **val_loss** has not changed more than **0.001** over the past **5 epochs**

In [ ]:
history_2 = best_model.fit(train_data,
                           validation_data = val_data,
                           epochs=60,
                           callbacks=[EarlyStopping(monitor='val_acc', min_delta=0.001, patience=5, verbose=1)]
                          )

Okay, our network stopped after **38** epochs as the **callback** fired.

Perhaps after a few epochs, **val_acc** would begin to grow, but this accuracy suits me, since this notebook was not written to find the best results, but to work out and understand working with CNN in Keras.

Let's take a look at the accuracy plot

In [ ]:
loss = history_2.history['acc']
val_loss = history_2.history['val_acc']

plt.figure(figsize=(15,8))
plt.plot(loss, label='Train acc')
plt.plot(val_loss,'--', label='Val acc')
plt.title('Training and validation accuracy')
plt.yticks(np.arange(0.5, 1, 0.05))
plt.xticks(np.arange(0, 26))
plt.grid()
plt.legend();

Compared to our first model, we really got rid of the **overfitting** problem. Now our metrics on the chart grow in proportion to each other

Let's save the model and apply it to the test data

In [ ]:
best_model.save('best_model_cat_vs_dog.h5')

#   <center>Result on test data</center>

First, we need to convert **test data** to the same form as **val** using a **generator**

**IMPORTANT!!!** It is necessary to set **shuffle=False** in order to avoid data mixing!!!

In [ ]:
test_data = val_gen.flow_from_dataframe(test,
                                        directory=path,
                                        x_col='name',
                                        y_col='label',
                                        class_mode='binary',
                                        target_size=(224,224),
                                        shuffle=False,
                                        seed=17  
                                       )

In [ ]:
test_pred = best_model.predict(test_data)

In [ ]:
pred_label = test_pred > 0.5
true_label = test_data.classes

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(true_label, pred_label), display_labels=test_data.class_indices).plot();

In [ ]:
best_model.evaluate(test_data)

Great, we have **95% on test data**

It is possible that the model could be trained for high accuracy results, however, the purpose of this notebook was to learn how to work with **Keras** and **CNN**


-----------------

![](https://aurora.ekof.bg.ac.rs/~s160748/331.jpg)

##  <center>Thank you for watching this is my project, I will be grateful if you upvoted and give feedback about my work in the comments. I want to improve my skills, and if you find any mistakes in the work, please tell me about it.</center>